# *In silico* evolution of promoters

## Import modules and define functions

Import the required modules:

In [1]:
import numpy as np
import pandas as pd
import re
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import tensorflow as tf
from tensorflow import keras as k

Enable GPU memory growth:

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs 1 Logical GPUs


Define a function to one-hot encode the DNA sequences (adapted from https://colab.research.google.com/drive/17E4h5aAOioh5DiTo7MZg4hpL6Z_0FyWr):

In [3]:
integer_encoder = LabelEncoder()  

one_hot_encoder = OneHotEncoder(categories='auto')

def one_hot_encoding(sequences, verbose = True): 
    one_hot_sequences = []

    if verbose:
        i = 0
        print('one-hot encoding in progress ...', flush = True)
    
    for sequence in sequences:
        integer_encoded = integer_encoder.fit_transform(list(sequence))
        integer_encoded = np.array(integer_encoded).reshape(-1, 1)
        one_hot_encoded = one_hot_encoder.fit_transform(integer_encoded)
        one_hot_sequences.append(one_hot_encoded.toarray())
    
        if verbose:
            i += 1
            if i % 1000 == 0:
                print(i, 'sequences processed', flush = True, end = '\r')
        
    if verbose:
        print('finished one-hot encoding:', i, 'sequences processed', flush = True)
    
    one_hot_sequences = np.stack(one_hot_sequences)

    return one_hot_sequences

Define a function to reverse the one-hot encoding:

In [4]:
def reverse_one_hot(one_hot_sequences, verbose = True):
    sequences = []
    
    if verbose:
        i = 0
        print('reversing one-hot encoding ...', flush = True)
    
    for one_hot_sequence in one_hot_sequences:
        integer_encoded = one_hot_encoder.inverse_transform(one_hot_sequence)
        integer_encoded = integer_encoded.flatten()
        sequence = integer_encoder.inverse_transform(integer_encoded)
        sequence = ''.join(sequence)
        sequences.append(sequence)
        
        if verbose:
            i += 1
            if i % 1000 == 0:
                print(i, 'sequences processed', flush = True, end = '\r')
                
    if verbose:
        print('finshed reverse one-hot encoding:', i, 'sequences processed', flush = True)
    
    return(sequences)

Define a function to generate all possible point mutations in a one-hot encoded sequence:

In [5]:
bases = one_hot_encoding(['ACGT'], verbose = False)
base_dict = dict(zip(['A', 'C', 'G', 'T'], bases[0]))

def mutate_seq(sequence):
    mutated_seqs = [sequence]
    for pos in range(len(sequence)):
        for base in base_dict.values():
            if not(np.array_equal(sequence[pos], base)):
                mut_seq = np.concatenate((sequence[:pos], base.reshape(1, -1), sequence[pos+1:]))
                mutated_seqs.append(mut_seq)
                
    mutated_seqs = np.stack(mutated_seqs)
    
    return(mutated_seqs)

Define a function to score sequences with models for both expression systems and pick the best sequence for the indicated system:

In [6]:
def score_seqs(sequences, system):
    prediction_leaf = model_leaf.predict(sequences)
    prediction_proto = model_proto.predict(sequences)
    if system == 'leaf':
        best_id = prediction_leaf.argmax()
    if system == 'proto':
        best_id = prediction_proto.argmax()
    if system == 'both':
        prediction_mean = (prediction_leaf + prediction_proto) / 2
        best_id = prediction_mean.argmax()
    
    return(sequences[best_id], prediction_leaf[best_id][0], prediction_proto[best_id][0])

Define a function to remove restriction enzyme sites from a sequence by mutating the site and picking the best performing sequence:

In [7]:
# Define restriction enzyme sites as regular expression.
# also considers sites that would be formed with the cloning adapter (5' G and 3' CC)
REsites = re.compile(r'((G|^)GTCT(C|$))|((G|^)AGA(CC|C$|$))|((G|^)AAGA(C|$))|((G|^)TCTT(C|$))')

# helper function to mutate sequences between 'start' and 'stop' 
def mutate_region(sequences, start, stop):
    mut_seqs = []
    
    for sequence in sequences:
        for pos in range(start, stop):
            for base in ['A', 'C', 'G', 'T']:
                if sequence[pos] != base:
                    mut_seq = sequence[:pos] + base + sequence[pos+1:]
                    mut_seqs.append(mut_seq)
                
    return (mut_seqs)

# function to find all matches to restriction enzyme sites, mutate them, and select the best performing sequence
def fix_REsite(sequence, system):
    matches = REsites.finditer(sequence)
    mut_seqs = [sequence]
    
    # introduce mutations to destroy RE sites
    for match in matches:
        mut_seqs = mutate_region(mut_seqs, match.start(), match.end())
        
    # filter out sequences where a new RE site was introduced
    mut_seqs = [seq for seq in mut_seqs if not REsites.search(seq)]
    
    # one-hot encode mutated sequences
    one_hot_seqs = one_hot_encoding(mut_seqs, verbose = False)
    
    # score sequences and pick best
    fixed_seq, leaf, proto = score_seqs(one_hot_seqs, system)
    
    # reverse one-hot encoding of fixed sequence
    fixed_seq = reverse_one_hot([fixed_seq], verbose = False)[0]
        
    return(fixed_seq, leaf, proto)

## Load models and prepare data

Load the models:

In [8]:
model_leaf = k.models.load_model('model_leaf')
model_proto = k.models.load_model('model_proto')

Load the data:

In [9]:
original_promoters = pd.read_csv('../analysis/validation_sequences/promoters_for_evolution.tsv', sep = '\t', header = 0)

One-hot encode start sequences:

In [10]:
start_seqs = one_hot_encoding(original_promoters['sequence'])

one-hot encoding in progress ...
finished one-hot encoding: 310 sequences processed


Predict promoter strength of start sequences:

In [11]:
prediction_leaf = model_leaf.predict(start_seqs)
prediction_proto = model_proto.predict(start_seqs)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity 

Set up dataframe for results:

In [12]:
evolution_data = pd.DataFrame(data = {
    'round' : 0,
    'origin' : original_promoters['name'],
    'opt_for' : 'start',
    'sequence' : list(start_seqs),
    'pred_leaf' : prediction_leaf.flatten(),
    'pred_proto' : prediction_proto.flatten()
})

## Perform the evolution

Perform iterative evolution of the promoter sequences:

In [13]:
n_rounds = 10

for system in ['leaf', 'proto', 'both']:
    print('optimizing promoters for:', system, flush = True)

    for thisround in range(1, n_rounds + 1):        
        lastround = thisround - 1
        data = evolution_data[evolution_data['round'] == lastround]

        if thisround == 1:
            data = data[data['opt_for'] == 'start']
        else:
            data = data[data['opt_for'] == system]
            
        if data.empty:
            break
            
        origins = data['origin']
        sequences = data['sequence']
            
        for origin, sequence in zip(origins, sequences):
            mut_seqs = mutate_seq(sequence)
            opt_seq, leaf, proto = score_seqs(mut_seqs, system)
            
            if not(np.array_equal(opt_seq, sequence)):
                evolution_data.loc[len(evolution_data)] = [thisround, origin, system, opt_seq, leaf, proto]
        
        print('finished round', thisround, flush = True, end = '\r')
        
    print('performed', thisround, 'rounds of evolution', flush = True)

optimizing promoters for: leaf
performed 10 rounds of evolution
optimizing promoters for: proto
performed 10 rounds of evolution
optimizing promoters for: both
performed 10 rounds of evolution


Reverse one-hot encoding:

In [14]:
evolution_data['sequence'] = reverse_one_hot(evolution_data['sequence'])

reversing one-hot encoding ...
finshed reverse one-hot encoding: 9610 sequences processed


Annotate if the sequence contains a restriction enzyme site:

In [15]:
evolution_data['REsite'] = [bool(REsites.search(x)) for x in evolution_data['sequence']]

Select sequences that need to be fixed (third or final round and contains a restriction enzyme site):

In [16]:
fix_seqs = evolution_data[((evolution_data['round'] == 3) | (evolution_data['round'] == max(evolution_data['round']))) & (evolution_data['REsite'])].copy()

Fix sequences:

In [17]:
fix_seqs[['sequence', 'pred_leaf', 'pred_proto']] = [fix_REsite(seq, sys) for (seq, sys) in zip(fix_seqs['sequence'], fix_seqs['opt_for'])]

Combine fixed sequences with the others:

In [18]:
evolution_data = fix_seqs.combine_first(evolution_data)

Convert 'round' column to integer:

In [19]:
evolution_data = evolution_data.astype({'round' : int})

Save data to a file

In [20]:
evolution_data.to_csv('evolution_data.tsv', sep = '\t', index = False)